In [15]:
import rasterio
from rasterio.windows import Window
from rasterio.warp import transform
import os
from math import ceil
from PIL import Image
import numpy as np

def analyze_and_tile_image(input_file="Ghaziabad2021.tif", tile_size_meters=200, output_dir="Ghaziabad2021"):
    """
    Modified version to create 150m x 150m tiles with filenames based on central pixel coordinates
    Fixed the import error for coordinate transformation
    """
    
    with rasterio.open(input_file) as src:
        # Extract actual resolution from the GeoTIFF metadata
        geo_transform = src.transform
        pixel_width = abs(geo_transform.a)   # meters per pixel in X direction
        pixel_height = abs(geo_transform.e)  # meters per pixel in Y direction
        
        # Get CRS information for coordinate transformation
        crs = src.crs
        
        # Get image dimensions in pixels
        width_pixels = src.width
        height_pixels = src.height
        
        # Calculate actual image dimensions in meters
        width_meters = width_pixels * pixel_width
        height_meters = height_pixels * pixel_height
        
        print("=== IMAGE ANALYSIS ===")
        print(f"CRS: {crs}")
        print(f"Resolution: {pixel_width}m/pixel × {pixel_height}m/pixel")
        print(f"Image dimensions (pixels): {width_pixels} × {height_pixels}")
        print(f"Image dimensions (meters): {width_meters:.1f}m × {height_meters:.1f}m")
        print(f"Geotransform: {geo_transform}")
        
        # Verify if resolution matches expected values
        if abs(pixel_width - 0.5) < 0.1:
            print("✓ Resolution confirmed: 0.5m/pixel (SkySat high-resolution)")
        elif abs(pixel_width - 1.0) < 0.1:
            print("✓ Resolution confirmed: 1.0m/pixel (SkySat standard)")
        else:
            print(f"⚠ Unexpected resolution: {pixel_width}m/pixel")
        
        # Calculate tile dimensions in pixels for 150m x 150m tiles
        tile_width_pixels = int(tile_size_meters / pixel_width)
        tile_height_pixels = int(tile_size_meters / pixel_height)
        
        # Calculate number of tiles needed
        x_tiles = ceil(width_pixels / tile_width_pixels)
        y_tiles = ceil(height_pixels / tile_height_pixels)
        
        print(f"\n=== TILING PLAN ===")
        print(f"Target tile size: {tile_size_meters}m × {tile_size_meters}m")
        print(f"Tile size in pixels: {tile_width_pixels} × {tile_height_pixels}")
        print(f"Number of tiles: {x_tiles} × {y_tiles} = {x_tiles * y_tiles} total tiles")
        
        # Create output directory
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Generate tiles
        print(f"\n=== CREATING TILES ===")
        tile_count = 0
        
        for y in range(y_tiles):
            for x in range(x_tiles):
                # Calculate window coordinates
                x_offset = x * tile_width_pixels
                y_offset = y * tile_height_pixels
                
                # Ensure we don't exceed image boundaries
                actual_width = min(tile_width_pixels, width_pixels - x_offset)
                actual_height = min(tile_height_pixels, height_pixels - y_offset)
                
                # Calculate the center pixel coordinates within the tile
                center_x_pixel = x_offset + actual_width // 2
                center_y_pixel = y_offset + actual_height // 2
                
                # Convert pixel coordinates to geographic coordinates
                # Using rasterio's transform to convert pixel to world coordinates
                center_x_world, center_y_world = geo_transform * (center_x_pixel, center_y_pixel)
                
                # If the CRS is not already lat/lon (WGS84), transform to WGS84
                if crs and crs.to_string() != 'EPSG:4326':
                    # Transform from current CRS to WGS84 (lat/lon) - FIXED VERSION
                    try:
                        # Create arrays for transformation
                        xs = [center_x_world]
                        ys = [center_y_world]
                        
                        # Use the correct transform function
                        lon_arr, lat_arr = transform(crs, 'EPSG:4326', xs, ys)
                        center_lon, center_lat = lon_arr[0], lat_arr[0]
                    except Exception as e:
                        print(f"Warning: CRS transformation failed: {e}")
                        print("Using original coordinates instead")
                        center_lon, center_lat = center_x_world, center_y_world
                else:
                    # Already in lat/lon coordinate system
                    center_lon, center_lat = center_x_world, center_y_world
                
                # Create window for reading the tile
                window = Window(x_offset, y_offset, actual_width, actual_height)
                
                # Read tile data
                tile_data = src.read(window=window)
                
                # Convert to format suitable for PIL (Height, Width, Channels)
                if tile_data.ndim == 3:
                    tile_data = tile_data.transpose(1, 2, 0)  # CHW to HWC
                
                # Convert to 8-bit for JPEG if needed
                if tile_data.dtype != np.uint8:
                    # Apply percentile stretch for better visualization
                    valid_data = tile_data[tile_data > 0]
                    if len(valid_data) > 0:
                        p2, p98 = np.percentile(valid_data, (2, 98))
                        tile_data = np.clip((tile_data - p2) / (p98 - p2) * 255, 0, 255)
                    tile_data = tile_data.astype(np.uint8)
                
                # Create PIL Image based on data dimensions
                if tile_data.ndim == 2:  # Grayscale
                    img = Image.fromarray(tile_data, mode='L')
                elif tile_data.shape[2] == 1:  # Single band
                    img = Image.fromarray(tile_data[:,:,0], mode='L')
                elif tile_data.shape[2] >= 3:  # RGB or multispectral
                    img = Image.fromarray(tile_data[:,:,:3], mode='RGB')
                
                # Generate filename with center coordinates (lat_lon format)
                # Using 6 decimal places for ~0.1m precision at equator
                tile_filename = f"{center_lat:.6f}_{center_lon:.6f}.jpg"
                tile_path = os.path.join(output_dir, tile_filename)
                
                # Save as high-quality JPEG
                img.save(tile_path, "JPEG", quality=95, optimize=True)
                tile_count += 1
                
                # Progress reporting with sample filenames
                if tile_count % 50 == 0 or tile_count == 1:
                    print(f"Progress: {tile_count}/{x_tiles * y_tiles} tiles created")
                    print(f"Sample filename: {tile_filename}")
        
        print(f"\n=== COMPLETED ===")
        print(f"Successfully created {tile_count} tiles of {tile_size_meters}m × {tile_size_meters}m")
        print(f"Output directory: {output_dir}")
        print(f"Filename format: [latitude]_[longitude].jpg (6 decimal places)")
        print(f"Coordinate system: WGS84 (EPSG:4326)")
        
        return {
            'resolution': (pixel_width, pixel_height),
            'image_size_pixels': (width_pixels, height_pixels),
            'image_size_meters': (width_meters, height_meters),
            'tiles_created': tile_count,
            'tile_grid': (x_tiles, y_tiles),
            'tile_size_meters': tile_size_meters,
            'crs': str(crs)
        }

# Run the analysis and tiling
result = analyze_and_tile_image()



=== IMAGE ANALYSIS ===
CRS: EPSG:32643
Resolution: 0.5m/pixel × 0.5m/pixel
Image dimensions (pixels): 2825 × 2971
Image dimensions (meters): 1412.5m × 1485.5m
Geotransform: | 0.50, 0.00, 735268.00|
| 0.00,-0.50, 3175296.00|
| 0.00, 0.00, 1.00|
✓ Resolution confirmed: 0.5m/pixel (SkySat high-resolution)

=== TILING PLAN ===
Target tile size: 200m × 200m
Tile size in pixels: 400 × 400
Number of tiles: 8 × 8 = 64 total tiles

=== CREATING TILES ===
Progress: 1/64 tiles created
Sample filename: 28.682593_77.408802.jpg
Progress: 50/64 tiles created
Sample filename: 28.671734_77.410600.jpg

=== COMPLETED ===
Successfully created 64 tiles of 200m × 200m
Output directory: Ghaziabad2021
Filename format: [latitude]_[longitude].jpg (6 decimal places)
Coordinate system: WGS84 (EPSG:4326)
